# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Detect-Dynamic-Communities" data-toc-modified-id="Detect-Dynamic-Communities-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Detect Dynamic Communities</a></div><div class="lev2 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev2 toc-item"><a href="#Aggregate-Adjacency-Matrices-across-Subjects" data-toc-modified-id="Aggregate-Adjacency-Matrices-across-Subjects-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Aggregate Adjacency Matrices across Subjects</a></div><div class="lev2 toc-item"><a href="#Run-GenLouvain-algorithm-using-MATLAB" data-toc-modified-id="Run-GenLouvain-algorithm-using-MATLAB-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Run GenLouvain algorithm using MATLAB</a></div><div class="lev1 toc-item"><a href="#Statistics-on-Dynamic-Modules" data-toc-modified-id="Statistics-on-Dynamic-Modules-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Statistics on Dynamic Modules</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev2 toc-item"><a href="#Module-Allegiance" data-toc-modified-id="Module-Allegiance-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Module Allegiance</a></div><div class="lev2 toc-item"><a href="#Find-clusters-in-module-allegiance-matrix" data-toc-modified-id="Find-clusters-in-module-allegiance-matrix-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Find clusters in module allegiance matrix</a></div>

# Detect Dynamic Communities

## Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_CoreData = '/Users/akhambhati/Remotes/CORE.fMRI_multiband.mmattar/restdata'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks'
path_InpData = path_PeriphData + '/e01-Dyne_FuncNetw'
path_ExpData = path_PeriphData + '/e02b-DynFuncModule-Population'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

## Generate List of Data

In [ ]:
subj_date = [full_subj_path.split('/')[-1]
             for full_subj_path in glob.iglob('{}/Adjacency.*.npz'.format(path_InpData))]

subj_ids = {}
for s_d in subj_date:
    subj, date = s_d.split('.')[:2]
    try:
        subj_ids[subj]
    except KeyError:
        subj_ids[subj] = []
    
    subj_ids[subj].append(date)

## Aggregate Adjacency Matrices across Subjects

In [ ]:
adj_matr = []
adj_name = []
for df_name in subj_date:
    
    # Read the input data
    df = np.load('{}/{}'.format(path_InpData, df_name))

    for adj in df['adj_matr']:
        adj_matr.append(adj)
        adj_name.append('.'.join(df_name.split('.')[1:3]))

# Cache the configuration matrices
adj_matr = np.array(adj_matr)
adj_name = np.array(adj_name)
io.savemat('{}/Module_Optimization.AdjMatr.mat'.format(path_ExpData),
           {'adj_matr': adj_matr, 
            'adj_name': adj_name})

## Run GenLouvain algorithm using MATLAB
Refer to "e02b_Module_Detection.m" for this subsection

# Statistics on Dynamic Modules

## Generate List of Data

In [ ]:
module_seed_path = glob.glob('{}/Module_Optimization.ModAssign.*.mat'.format(path_ExpData))

## Module Allegiance

In [ ]:
%matplotlib inline

for pth_i, pth in enumerate(np.random.permutation(module_seed_path)):
    df = io.loadmat(pth)
    
    S_subj = df['Ssubj']
    n_subj = S_subj.shape[1]

    
    # Plot the Time-Varying Modules for one seed
    if pth_i==0:
        plt.figure()
        n_rows = int(np.sqrt(n_subj))
        n_cols = int(np.ceil(n_subj/n_rows))
        for ii in xrange(n_subj):
            ax = plt.subplot(n_rows, n_cols, ii+1)
            ax.matshow(S_subj[0, ii], cmap='rainbow')
            ax.set_axis_off()
        plt.show()

     # Compute Allegiance   
    for ii in xrange(n_subj):
        module_assign = S_subj[0, ii].T
        
        n_win = module_assign.shape[0]
        n_node = module_assign.shape[1] 
        
        if (pth_i == 0) and (ii == 0):
            module_alleg = np.zeros((n_node, n_node))
            
        for iw in xrange(n_win):
            module_id = np.unique(module_assign[iw, :])
            
            for m_id in module_id:
                m_nodes = np.flatnonzero(module_assign[iw, :] == m_id)
                n1, n2 = np.meshgrid(m_nodes, m_nodes)
                
                module_alleg[n1, n2] += 1
module_alleg /= module_alleg[0, 0]
module_alleg[np.diag_indices_from(module_alleg)] = 0
np.savez("{}/consensus_module_allegiance.npz".format(path_ExpData),
         module_alleg=module_alleg) 

# Plot the Module Allegiance across the cohort
plt.figure()
ax = plt.subplot(111)
mat = ax.matshow(module_alleg, vmin=0, vmax=1, cmap='rainbow')
plt.colorbar(mat, ax=ax)
ax.set_axis_off()
ax.set_title('Population Module Allegiance')

## Find clusters in module allegiance matrix

In [ ]:
# Compute Static Modularity
df = np.load('{}/consensus_module_allegiance.npz'.format(path_ExpData, subj))
module_alleg = df['module_alleg']
k = np.sum(module_alleg, axis=0)
twom = np.sum(k)

qq = []
gamma = np.linspace(0.5, 3.0, 200)
all_mod_id = []
for gm in gamma:
    B = module_alleg - gm*np.dot(k.reshape(-1, 1), k.reshape(1, -1)) / twom

    mod_alleg_id, q = Echobase.Network.Partitioning.Module.community.genlouvain(B,
                                                                                limit=10000,
                                                                                verbose=False)
    qq.append(q)
    all_mod_id.append(mod_alleg_id)
qq = np.array(qq)

opt_ix = Echobase.Statistics.Optimization.curve.find_elbow(qq)
true_mod_alleg = all_mod_id[opt_ix]

# Optimization Plot
plt.figure()
ax = plt.subplot(111)
ax.plot(gamma, qq)
ax.scatter(gamma[opt_ix], qq[opt_ix], marker='x', color='r')

# Axis Settings
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.set_xlabel('Gamma');
ax.set_ylabel('Modularity');
plt.show()

# Re-ordered Module Allegiance
plt.figure()
ax = plt.subplot(111)
mat = ax.matshow(module_alleg[np.meshgrid(np.argsort(true_mod_alleg), np.argsort(true_mod_alleg))],
                 cmap='rainbow', vmin=0, vmax=1);
plt.colorbar(mat, ax=ax)

# Axis Settings
ax.set_axis_off()
ax.set_xlabel('Brain Regions');
ax.set_ylabel('Brain Regions');
plt.show()

# Print out the module membership assignment
roi_csv = pd.read_csv('{}/Atlas/HOA112_Labels.csv'.format(path_CoreData))
for module_id in np.unique(true_mod_alleg):
    mod_ix = np.flatnonzero(true_mod_alleg == module_id)
    
    print('Module_ID: {}'.format(module_id+1))     
    print(np.unique(roi_csv['Nomenclature'][mod_ix]))